# 96_fig6_asymmetry

Figure 6: asymmetry normal→bad vs bad→normal (commitment) using forced regime paths.

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
sys.path.append("..")

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = os.path.join("..","artifacts","runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

# --- paper reporting helpers ---
ann = lambda x: 400.0*x  # annualized percent (quarterly -> annual)


In [ ]:
from src.steady_states import solve_commitment_sss_from_policy_switching
from src.experiments import DeterministicPathSpec, simulate_deterministic_path

run_dir = get_run("commitment")
params = load_params_from_run(run_dir)
net = load_net_from_run(run_dir, 7, 13)
sss = solve_commitment_sss_from_policy_switching(params, net)

x0_n = torch.tensor([[float(sss.by_regime[0]["Delta_prev"]), float(sss.by_regime[0]["logA"]), float(sss.by_regime[0]["loggtilde"]), float(sss.by_regime[0]["xi"]), 0.0,
                      float(sss.by_regime[0]["vartheta_prev"]), float(sss.by_regime[0]["varrho_prev"])]], dtype=torch.float32)
x0_b = torch.tensor([[float(sss.by_regime[1]["Delta_prev"]), float(sss.by_regime[1]["logA"]), float(sss.by_regime[1]["loggtilde"]), float(sss.by_regime[1]["xi"]), 1.0,
                      float(sss.by_regime[1]["vartheta_prev"]), float(sss.by_regime[1]["varrho_prev"])]], dtype=torch.float32)

T=40
path_nb = simulate_deterministic_path(params, "commitment", net, x0=x0_n,
                                     spec=DeterministicPathSpec(T=T, regime_path=[0]+[1]*T),
                                     compute_implied_i=True)
path_bn = simulate_deterministic_path(params, "commitment", net, x0=x0_b,
                                     spec=DeterministicPathSpec(T=T, regime_path=[1]+[0]*T),
                                     compute_implied_i=True)

# regime_path applies to s_{t+1}; use index 1 as aligned impact t=0.
t=np.arange(T)
pi_nb = path_nb["pi"][1:,0]
pi_bn = path_bn["pi"][1:,0]

plt.figure()
plt.plot(t, ann(pi_nb), label="normal->bad")
plt.plot(t, -ann(pi_bn), label="-(bad->normal)", linestyle="--")
plt.axhline(0, linewidth=1)
plt.title("Figure 6: asymmetry in pi (sign-reversed)")
plt.xlabel("t"); plt.ylabel("pi"); plt.legend(); plt.show()
